In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from underthesea import word_tokenize
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import random
import tensorflow as tf

# Nhập dữ liệu

In [2]:
# Tải dữ liệu
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

# Gắn nhãn cho bộ dữ liệu

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'Tiêu cực'
    elif rating == 3:
        return 'Trung tính'
    elif rating in [4, 5]:
        return 'Tích cực'
    else:
        return 'Không xác định'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      Điện thoại này dùng rất thích    Tích cực
1            4                               sử dụng thấy cũng ok    Tích cực
2            2                      Bảo hành ít quá, chỉ 12 tháng    Tiêu cực
3            5                             Sản phẩm mượt, chạy êm    Tích cực
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...  Trung tính


# Tiền xử lý dữ liệu

In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt  Tích cực
76              5                                    Rất ok  Tích cực
107             2                             pin tụt nhanh  Tiêu cực
135             5                              sản phẩm tốt  Tích cực
173             2                             hao pin nhanh  Tiêu cực
177             5                              sản phẩm tốt  Tích cực
181             5                                    Rất ok  Tích cực
183             5                              sản phẩm tốt  Tích cực
192             5                               sản phẩm ok  Tích cực
205             5                               máy dùng ok  Tích cực
225             4                                     

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      điện thoại này dùng rất thích    Tích cực
1            4                               sử dụng thấy cũng ok    Tích cực
2            2                       bảo hành ít quá chỉ 12 tháng    Tiêu cực
3            5                              sản phẩm mượt chạy êm    Tích cực
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...  Trung tính


In [6]:
# Lưu đánh giá vào cột X
X = df['Comment'].values.tolist()
# Lưu nhãn vào cột Y
y = df['Sentiment'].values.tolist()

# Tách từ, xây dựng bộ từ vựng và mã hóa

In [7]:
#Xây dựng hàm token từ underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

In [8]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_build_vocab_vietnamese, token_pattern=None,
                              max_features=5000, ngram_range=(1, 2), max_df=0.85, min_df=5)
# Áp dụng với cột Comment
X_tfidf = vectorizer.fit_transform(X)
vocabulary = vectorizer.get_feature_names_out()

# Mã hóa cho cột nhãn

In [9]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Cân bằng dữ liệu

In [10]:
#Sử dụng SMOTE để cân bằng dữ liệu

print('Trước khi cân bằng:', Counter(y))

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

print('Sau khi cân bằng:', Counter(y_resampled))


Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [11]:
print('Kích thước dữ liệu ban đầu:', X_tfidf.shape)
print('Kích thước dữ liệu sau khi cân bằng:', X_resampled.shape)
# In ra số lượng mẫu của từng nhãn trước và sau khi cân bằng
print('Trước khi cân bằng:', Counter(y))
print('Sau khi cân bằng:', Counter(y_resampled))

Kích thước dữ liệu ban đầu: (1498, 1200)
Kích thước dữ liệu sau khi cân bằng: (2496, 1200)
Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [12]:
X_tfidf = X_tfidf.toarray()
# Xóa dòng mã trên, và thay thế nó bằng dòng mã sau:
X_lstm = X_tfidf.reshape(X_tfidf.shape[0], X_tfidf.shape[1], 1)

In [15]:
# Reshape TF-IDF matrix for LSTM input
X_lstm = X_tfidf.toarray()  # Ensure X_tfidf is already TF-IDF transformed and converted to array

# Reshape to (number_of_samples, max_features, 1) for LSTM input
X_lstm = np.expand_dims(X_lstm, axis=-1)


AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

# Chia tập dữ liệu

In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

# Xây dựng mô hình LSTM

In [14]:
# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=X_train.shape[1], output_dim=128, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 1200, 1, 128)

In [ ]:
# Train model
epochs = 10
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
47/47 [==============================] - 3s 24ms/step - loss: 1.0373 - accuracy: 0.6119 - val_loss: 0.8980 - val_accuracy: 0.6987
Epoch 2/5
47/47 [==============================] - 1s 12ms/step - loss: 0.6448 - accuracy: 0.8156 - val_loss: 0.4599 - val_accuracy: 0.8560
Epoch 3/5
47/47 [==============================] - 0s 8ms/step - loss: 0.2888 - accuracy: 0.9158 - val_loss: 0.3198 - val_accuracy: 0.8853
Epoch 4/5
47/47 [==============================] - 0s 10ms/step - loss: 0.1460 - accuracy: 0.9613 - val_loss: 0.3031 - val_accuracy: 0.8933
Epoch 5/5
47/47 [==============================] - 0s 8ms/step - loss: 0.0828 - accuracy: 0.9773 - val_loss: 0.2960 - val_accuracy: 0.9040


In [ ]:
#Đánh giá mô hình
score = model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

20/20 [==============================] - 0s 3ms/step - loss: 0.2564 - accuracy: 0.9167
Test Loss: 0.25642144680023193
Test Accuracy: 0.9166666865348816


# Tìm tham số tối ưu hyperparameters

In [ ]:
# Đặt seed cho tất cả các nguồn ngẫu nhiên để đảm bảo rằng mỗi lần chạy đều ra kết quả giống nhau
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
# Hàm xây dựng mô hình cho GridSearch
def create_model(neurons, dropout_rate, optimizer):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(neurons // 2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Xây dựng mô hình KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Tham số tìm kiếm GridSearch
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [5,10, 20, 30],
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [64, 128],
    'dropout_rate': [0.2, 0.3]
}

# GridSearch
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Hiển thị kết quả
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{mean} ({std}) with: {param}")


C:\Users\vuhamy\AppData\Local\Temp\ipykernel_13956\2553851434.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.8684034744898478 using {'batch_size': 32, 'dropout_rate': 0.3, 'epochs': 10, 'neurons': 64, 'optimizer': 'adam'}
0.857047438621521 (0.03216138437832708) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 64, 'optimizer': 'adam'}
0.8390113711357117 (0.026094892004427037) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 64, 'optimizer': 'rmsprop'}
0.8590514461199442 (0.029331314483772245) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 128, 'optimizer': 'adam'}
0.8563794294993082 (0.02315955098328221) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 5, 'neurons': 128, 'optimizer': 'rmsprop'}
0.8643954594930013 (0.028917632927458248) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 10, 'neurons': 64, 'optimizer': 'adam'}
0.8610554536183676 (0.027976477175872365) with: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 10, 'neurons': 64, 'optimizer': 'rmsprop'}
0.862391451994578 (0.02629930573082417) with: {

In [ ]:
# Lấy tham số tốt nhất
best_params = grid_result.best_params_

# Tạo mô hình mới với tham số tốt nhất
final_model = create_model(neurons=best_params['neurons'],
                           optimizer=best_params['optimizer'],
                           dropout_rate=best_params['dropout_rate'])

# Huấn luyện mô hình mới
final_model.fit(X_train, y_train,epochs=best_params['epochs'], batch_size=best_params['batch_size'], 
                          validation_data=(X_val, y_val), 
                          callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/10
47/47 [==============================] - 7s 25ms/step - loss: 1.0580 - accuracy: 0.4603 - val_loss: 0.9691 - val_accuracy: 0.7067
Epoch 2/10
47/47 [==============================] - 0s 7ms/step - loss: 0.8105 - accuracy: 0.7869 - val_loss: 0.6391 - val_accuracy: 0.8613
Epoch 3/10
47/47 [==============================] - 0s 7ms/step - loss: 0.4631 - accuracy: 0.8898 - val_loss: 0.4121 - val_accuracy: 0.8827
Epoch 4/10
47/47 [==============================] - 0s 7ms/step - loss: 0.2680 - accuracy: 0.9305 - val_loss: 0.3277 - val_accuracy: 0.8907
Epoch 5/10
47/47 [==============================] - 0s 9ms/step - loss: 0.1658 - accuracy: 0.9619 - val_loss: 0.3009 - val_accuracy: 0.9147
Epoch 6/10
47/47 [==============================] - 1s 12ms/step - loss: 0.1097 - accuracy: 0.9806 - val_loss: 0.3030 - val_accuracy: 0.9093
Epoch 7/10
47/47 [==============================] - 0s 7ms/step - loss: 0.0752 - accuracy: 0.9846 - val_loss: 0.3032 - val_accuracy: 0.9067
Epoch 8/10
47/47 [

# Kiểm thử và dự đoán mô hình

In [ ]:
# Đánh giá mô hình với tham số mới
score = final_model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

20/20 [==============================] - 0s 5ms/step - loss: 0.2552 - accuracy: 0.9199
Test Loss: 0.25520509481430054
Test Accuracy: 0.9198718070983887


In [ ]:
# Đánh giá mới cần kiểm thử
new_reviews = [
    "Sản phẩm tốt, rất hài lòng!",
    "Pin yếu, cam tệ, màn hình giật lag liên tục",
    "Chỉ sử dụng ở mức tạm ổn"
]

# Tiền xử lý đánh giá mới
def preprocess_new_reviews(reviews):
    preprocessed_reviews = [normalize_text(review) for review in reviews]
    return preprocessed_reviews

preprocessed_reviews = preprocess_new_reviews(new_reviews)

# Chuyển đổi đánh giá mới sang TF-IDF
new_reviews_tfidf = vectorizer.transform(preprocessed_reviews).toarray()

# Dự đoán nhãn cho đánh giá mới
predictions = final_model.predict(new_reviews_tfidf)
predicted_labels = np.argmax(predictions, axis=1)

# Chuyển đổi nhãn dự đoán từ số sang tên
predicted_sentiments = label_encoder.inverse_transform(predicted_labels)

# In kết quả
for review, sentiment in zip(new_reviews, predicted_sentiments):
    print(f"Đánh giá: {review}")
    print(f"Dự đoán cảm xúc: {sentiment}")
    print()

# Báo cáo đánh giá mô hình trên tập test
y_pred = final_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))


1/1 [==============================] - 0s 408ms/step
Đánh giá: Sản phẩm tốt, rất hài lòng!
Dự đoán cảm xúc: Tích cực

Đánh giá: Pin yếu, cam tệ, màn hình giật lag liên tục
Dự đoán cảm xúc: Tiêu cực

Đánh giá: Chỉ sử dụng ở mức tạm ổn
Dự đoán cảm xúc: Trung tính

20/20 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

    Tiêu cực       0.92      0.92      0.92       209
  Trung tính       0.87      0.96      0.91       204
    Tích cực       0.97      0.88      0.92       211

    accuracy                           0.92       624
   macro avg       0.92      0.92      0.92       624
weighted avg       0.92      0.92      0.92       624

